In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#default_exp nia.woa

# Whale Optimization Algorithm

    Implementation of WOA (2016)

In [ ]:
#export
from PrimeCNNv3.imports import *

In [ ]:
#export

class Particle:
    pass

In [ ]:
#export
class WOA:
    def __init__(self,a, a_step, b, population, fitness_eval_func,n_dims, dim_limits, maximize = False):
        '''
            input:
                    
                    population: total number of solutions to generate
                    fintess_eval_func: fitness evaluation function
                    n_dims: number of dimensions for the solution
                    dim_limit (dictionary): keys being the index for which the limit i.e value of the key is applicable
                    
                    a_steps = a / iters
                    a = 2.0
                    b = 0.5
                    
                    maximize if True, problem is of maximization else minimization
                    
            
            ref:
              Seyedali Mirjalili, Andrew Lewis,
              The Whale Optimization Algorithm,
              Advances in Engineering Software,
              Volume 95,
              2016,
              Pages 51-67,
              ISSN 0965-9978,
              https://doi.org/10.1016/j.advengsoft.2016.01.008.
              (https://www.sciencedirect.com/science/article/pii/S0965997816300163)
              Abstract: This paper proposes a novel nature-inspired meta-heuristic optimization algorithm, called Whale Optimization Algorithm (WOA), which mimics the social behavior of humpback whales. The algorithm is inspired by the bubble-net hunting strategy. WOA is tested with 29 mathematical optimization problems and 6 structural design problems. Optimization results prove that the WOA algorithm is very competitive compared to the state-of-art meta-heuristic algorithms as well as conventional methods. The source codes of the WOA algorithm are publicly available at http://www.alimirjalili.com/WOA.html
              Keywords: Optimization; Benchmark; Constrained optimization; Particle swarm optimization; Algorithm; Heuristic algorithm; Genetic algorithm; Structural optimization
        '''
        
        self._a = a
        self._a_step = a_step
        self._b = b
        self.population = population
        self.fitness_eval_func = fitness_eval_func
        self.n_dims = n_dims
        self.dim_limits = dim_limits
        self.best_solutions = []
        self.Swarm = []
        self.maximize = maximize
        
    def _initialize_population(self):
        '''Initialize the whale population'''
        for solution in range(self.population):
            start = 0
            whale = Particle()
            whale.position = np.array([])
            for end in self.dim_limits:
                vec = np.random.uniform(*self.dim_limits[end], (end-start))
                whale.position = np.concatenate((whale.position, vec))
                start = end
            
            whale.fitness = self.fitness_eval_func(whale.position)
            
            self.Swarm.append(whale) 
            
        #sort according to fitness
        self._sort_fitness()
        
        #add best solution i.e keep track best solution is at top
        self.best_solutions.append(self.Swarm[0])
       

  
    def optimize(self):
       
        # prey = best solution
        prey = self.Swarm[0]
        #print(f'Best solution {prey.position}')
        for whale in self.Swarm[1:]:
            if np.random.uniform() > 0.5:
                A = self._compute_A()
                abs_A = np.linalg.norm(A)
                
                if abs_A < 1.0:
                    whale.position = self._encircle_prey(whale_pos = whale.position, prey_pos = prey.position, A = A)
                else:
                    random_prey = self.Swarm[np.random.randint(1, self.population)]
                    whale.position = self._search_prey(whale_pos = whale.position, random_prey_pos = random_prey.position, A = A)
            else:
                whale.position = self._attack(whale_pos = whale.position, prey_pos = prey.position)
            
            #constraint the whales limits
            whale.position = self._constraint_solutions(whale.position)
            
            #calculate fitness
            whale.fitness = self.fitness_eval_func(whale.position)
        
        self._a -= self._a_step
        
        #sort according to fitness
        self._sort_fitness()
        
        #add best solution i.e keep track best solution is at top
        self.best_solutions.append(self.Swarm[0])
        
    
    def _constraint_solutions(self, position):
        start = 0
        for end in self.dim_limits:
            position[start:end] = np.maximum(position[start:end], self.dim_limits[end][0]) #minimum limit
            position[start:end] = np.minimum(position[start:end], self.dim_limits[end][1]) # maximum limit
            
            start = end
            
        return position
    
    def _encircle_prey(self, whale_pos, prey_pos, A):
        '''
            Exploitation phase
            Update position according to the prey i.e Shrinking encircling Mechanism
            
            Xi_new = X* - (A . D)....e.q 2.2
            
            where
                X* is Best solution
                
        '''
        D = self._compute_D(prey_pos = prey_pos, whale_pos = whale_pos)
        return prey_pos - (A * D)
    
    def _search_prey(self,whale_pos,random_prey_pos, A):
        '''
            Exploration phase
            return new position
            
            Xi_new = X_rand - (A . D)
            
            where
                X_rand is random search agent 
        
        '''
        D = self._search_D(random_prey_pos = random_prey_pos, whale_pos = whale_pos)
        return random_prey_pos - (A * D)
    
    def _attack(self,whale_pos, prey_pos):
        '''
            Exploitation phase
            Spiral updating phase
            
            prey = Best search Agent
            
            D = |X* - Xi|      ....e.q. 2.5
            L = random number betwwen -1.0 to 1.0    ....e.q. 2.5
            Xi_new = (D . (exp^(b*L) . cos(2*pi*L))) + X*   .... e.q. 2.5
        '''
        D = np.abs(prey_pos - whale_pos)
        L = np.random.uniform(-1.0, 1.0)
        
        return (D * (np.exp(self._b * L) * np.cos(2.0 * np.pi * L))) + prey_pos
    
    def _sort_fitness(self):
        
        self.Swarm.sort(key = lambda prey:prey.fitness, reverse = self.maximize)
        
    
    
    def _compute_A(self):
        '''
            A_vec = (2.0 * (a . r_vec)) - a  ....e.q 2.3
            where:
                r_vec is random vector between 0,1
        '''
        r_vec = np.random.uniform(size = self.n_dims)
        return (2.0*(self._a * r_vec)) - self._a
    
    def _compute_C(self):
        '''
            C_vec = 2 . r_vec    ......e.q 2.4
            
            where:
                 r_vec is random vector between 0,1
        '''
        r_vec = np.random.uniform(size = self.n_dims)
        return 2.0 * r_vec
    
    def _compute_D(self, prey_pos, whale_pos):
        '''
            prey: if best_soltion ---> encircle_d 
                  if random_solution ---> search_d
            whale: that encircles the prey
            
            D = |(C_vec . X*) - Xi| ....e.q  2.1
            where 
                X* is the best solution
                Xi current search agent
        '''
        C = self._compute_C()
        return np.abs((C*prey_pos) - whale_pos)
  
    def _search_D(self,random_prey_pos, whale_pos):
        '''
            D = |(C_vec . X_rand) - Xi|  .... e.q 2.7
            where 
                X_rand is the Random search agent
                Xi current search agent
        '''
        return self._compute_D(prey_pos = random_prey_pos, whale_pos = whale_pos)
    

In [ ]:
from nbdev.export import notebook2script; notebook2script()

Converted 00_utils.data.dataset.ipynb.
Converted 01_utils.data.dataloaders.ipynb.
Converted 02_utils.vizualize.ipynb.
Converted 03_callbacks.ipynb.
Converted 03a_hooks.ipynb.
Converted 04_learner.ipynb.
Converted 05_metrics.ipynb.
Converted 06_NIA.PSO.ipynb.
Converted 06_NIA.WOA.ipynb.
Converted index.ipynb.
